# PV Degradation Tools Tutorial - 1
### Relative Humidity and Edge Seal Width
***
Matthew Brown

2023.06.02
***

**Requirements:**
- weather file (psm3 preferred) demo is provided

**Objectives:**
1. Read in necessary weather data
2. Generate POA and Solar Position
3. Generate module humidities
4. Calculate edge seal width

In [2]:
import os
import numpy as np
import pandas as pd

import pvdeg 
from pvdeg import TEST_DATA_DIR

## 1.
From a psm3 (or TMY3 or EPW) file we need the following:
- DNI
- DHI
- GHI
- Temperature, Dry-Bulb
- Temperature, Dew-Point
- Wind Speed
- Relative Humidity
- Latitude
- Longitude
- altitude

In [3]:
PSM_FILE = os.path.join(TEST_DATA_DIR,'psm3_pytest.csv')
WEATHER, META = pvdeg.weather.read(PSM_FILE,'psm')

dni = WEATHER.dni
dhi = WEATHER.dhi
ghi = WEATHER.ghi

time_range = WEATHER.index

In [6]:
WEATHER

,Year,Month,Day,Hour,Minute,dni,dhi,ghi,temp_air,Dew Point,wind_speed,relative_humidity,poa_global,temp_cell,temp_module
1999-01-01 00:30:00-07:00,1999,1,1,0,30,0.0,0.0,0.0,0.0,-5.0,1.8,79.39,0.0,0.0,0.0
1999-01-01 01:30:00-07:00,1999,1,1,1,30,0.0,0.0,0.0,0.0,-4.0,1.7,80.84,0.0,0.0,0.0
1999-01-01 02:30:00-07:00,1999,1,1,2,30,0.0,0.0,0.0,0.0,-4.0,1.5,82.98,0.0,0.0,0.0
1999-01-01 03:30:00-07:00,1999,1,1,3,30,0.0,0.0,0.0,0.0,-4.0,1.3,85.01,0.0,0.0,0.0
1999-01-01 04:30:00-07:00,1999,1,1,4,30,0.0,0.0,0.0,0.0,-4.0,1.3,85.81,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999-12-31 19:30:00-07:00,1999,12,31,19,30,0.0,0.0,0.0,0.0,-3.0,0.9,83.63,0.0,0.0,0.0
1999-12-31 20:30:00-07:00,1999,12,31,20,30,0.0,0.0,0.0,0.0,-3.0,1.2,86.82,0.0,0.0,0.0
1999-12-31 21:30:00-07:00,1999,12,31,21,30,0.0,0.0,0.0,0.0,-4.0,1.6,83.78,0.0,0.0,0.0
1999-12-31 22:30:00-07:00,1999,12,31,22,30,0.0,0.0,0.0,0.0,-4.0,1.7,81.22,0.0,0.0,0.0


## 2. Solar Position and POA

Next we need to generate the solar position (azimuth, elevation, etc) and plane of array irradiance (POA). `PVDEG` has a wrapper for quickly using `PVLIB` to generate these figures.

In [4]:
sol_pos = pvdeg.spectral.solar_position(weather_df=WEATHER, meta=META)

poa_df = pvdeg.spectral.poa_irradiance(weather_df=WEATHER, meta=META, solar_position=sol_pos)

# for now, all we need is the POA_GLOBAL, so grab the column from the dataframe
poa_global = poa_df.poa_global

## 3. Module Humidities
`pvdeg` can be used to calculate the relative humidity of several layers within a PV module. This can be done with 2 techniques.

a. Automatic
    
b. Manual
    

### 3.a. Automatic
Use the function `calc_rel_humidity`. This method does all of the calculations behind the scenes. It requires only 2 inputs (weather data and meta data)

In [5]:
rh_module = pvdeg.humidity.calc_rel_humidity(weather_df=WEATHER, meta=META)

KeyError: 'air_temperature'

### 3.b. Manual
Use the individual functions to calculate the RH in each layer. These are named and require more extensive input parameters. This must be done in the correct order:
1. front encapsulant
2. back encapsulant
3. backsheet

In [ ]:
rh_front_encap = pvdeg.StressFactors.rh_front_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_back_encap = pvdeg.StressFactors.rh_back_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_backsheet = pvdeg.StressFactors.rh_backsheet_from_encap(rh_back_encap=rh_back_encap,
                                                         rh_surface_outside=data['Relative Humidity'])

## 4. 
### Design: Edge Seal Width
`pvdeg.design.edge_seal_width` calculates the width [cm] required for several years of water ingress. If you do not specify the number of years, it will default to 25 years.

In [4]:
psat, psat_avg = pvdeg.humidity.psat(WEATHER['temp_air'])

k = pvdeg.design.k(avg_psat=psat_avg)

edge_seal_width = pvdeg.design.edge_seal_width(k, years=20)

print(psat_avg)
print(k)
print(edge_seal_width)

1.3946499092628233
0.00153182106297676
0.6413923384461668


### Alternate Edge Seal Width
If you have the dew point, or wet-bulb temperature, you can calculate the edge seal width directly.

In [6]:
edge_seal_width = pvdeg.design.edge_seal_from_dew_pt(dew_pt_temp=WEATHER['Dew Point'], years=20)
edge_seal_width

0.40245072298649226

### 6. Material Relative Humidity

In [18]:
rh_front_encap = pvdeg.StressFactors.rh_front_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_back_encap = pvdeg.StressFactors.rh_back_encap(rh_ambient=data['Relative Humidity'],
                                                    temp_ambient=data['Temperature'],
                                                    temp_module=temp_module)

rh_backsheet = pvdeg.StressFactors.rh_backsheet_from_encap(rh_back_encap=rh_back_encap,
                                                         rh_surface_outside=data['Relative Humidity'])

In [19]:
print(rh_front_encap[17])
print(rh_back_encap[17])
print(rh_backsheet[17])


50.28910527924019
80.45763598088315
81.22381799044157
